# ローカルユーザを新規登録します
<HR>

ローカルユーザを登録します。登録時にユーザのログインパスワードを自動生成します。<br/>
ローカルユーザとは、このコースにのみログインできるユーザです。SSOには利用できません。<br/>

次のセルで、下記のように登録するユーザのメールアドレスを配列`LOCAL_USERS_ADD_LIST`にを設定して、セルを実行して下さい。

```
LOCAL_USERS_ADD_LIST = '''
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''
```

**登録したユーザは、メールアドレスとパスワードで認証されます。**

**ユーザのメールアドレスとパスワードが講師環境の `admin_tools/data/students.tsv` に登録されますので、必ず、記録して下さい。**

In [ ]:
# 以下に、登録するローカルユーザのメールアドレスを列挙してください。
LOCAL_USERS_ADD_LIST = '''#ここから#
# GROUP A
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''#ここまで#

# ローカルユーザの追加

In [ ]:
import os
target_hub = ['-i', os.path.expanduser('~/ansible/inventory'), 'ch-master']

!ansible -m ping {' '.join(target_hub)}

**上記セルが動作しない場合**

```
The authenticity of host 'xxx.xxx.205.128 (xxx.xxx.205.128)' can't be established.
ECDSA key fingerprint is SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w.
Are you sure you want to continue connecting (yes/no)?
```
（IPアドレスと、fingerprintは例です）

となり実行中のまま状態変化しなくなる場合は、JupyterのTerminalから、

```
$ ssh xxx.xxx.205.128
```

を実行し、ECDSA key fingerprintが `SHA256:qjPDx7y/926gHJL9+SgMGKpicRORzffk1/xiUyIP00w` であることを確認してyesを実行し、上記のセルを停止の上再実行してください。

In [ ]:
import subprocess

default_user = !whoami
default_user = default_user[0]

result = subprocess.check_output(['ansible', '-m', 'ping'] + target_hub)
print(result.decode('utf8'))

In [ ]:
import re

student_mails = []
for line in LOCAL_USERS_ADD_LIST.splitlines():
    s = re.sub(r'#.*$', '', line).strip()
    if 0 < len(s):
        student_mails.append(line)
student_mails

In [ ]:
import hashlib

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

get_username_from_mail_address(student_mails[0])

ユーザをDBに登録します。パスワードは登録時に生成されます。

In [ ]:
def get_auth_proxy():
    import subprocess
    try:
        sid = subprocess.check_output(['ansible', '-b', '-a', 'docker service ps coursewarehub_auth-proxy -q'] + target_hub)
        sid = sid.decode('utf-8').split('\n')[1].strip()
        cinfo = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker inspect --format "{% raw %} {{.NodeID}} {{.Status.ContainerStatus.ContainerID}} {% endraw %}" ' + sid
            ] + target_hub)
        nodeid, cid = cinfo.decode('utf-8').split('\n')[1].strip().split()
        nodeip = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker node inspect --format "{% raw %} {{.Status.Addr}} {% endraw %}" ' + nodeid
            ] + target_hub)
        nodeip = nodeip.decode('utf-8').split('\n')[1].split()[0]
        return (nodeip, cid)
    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
        raise

auth_proxy = get_auth_proxy()
auth_proxy

In [ ]:
target_auth_proxy = ['-i', os.path.expanduser('~/ansible/inventory'), auth_proxy[0]]

local_user_infos = []
for student_mail in student_mails:
    local_user_info = subprocess.check_output(['ansible', '-b', '-a',
                                               'docker exec -i {} php /var/www/bin/add-local-user.php "{}"'.format(auth_proxy[1], student_mail)] + target_auth_proxy)
    local_user_infos.append(local_user_info.decode('utf8'))
local_user_infos[0]

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

ユーザ情報を控える。

ユーザのID, パスワードは、講師環境の `admin_tools/data/students.tsv` から確認することができます。

In [ ]:
r = subprocess.check_output(['ansible', '-b', '-m', 'file', '-a',
                             'path=/jupyter/admin/admin_tools/data/ state=directory'] + target_hub)
print(r.decode('utf8'))

for student_mail, local_user_info in zip(student_mails, local_user_infos):
    print(student_mail)
    student_id = get_username_from_mail_address(student_mail)
    with open(os.path.join(work_dir, student_id + '.tsv'), 'w') as f:
        f.write(local_user_info.split('\n')[1] + '\n')
        
    r = subprocess.check_output(['ansible', '-m', 'copy', '-a',
                                 'src={work_dir}/{student_id}.tsv dest=/jupyter/users/{default_user}/{student_id}.tsv'.format(work_dir=work_dir,
                                                                                                                              student_id=student_id,
                                                                                                                              default_user=default_user)] + target_hub)
    print(r.decode('utf8'))

    r = subprocess.check_output(['ansible', '-b', '-m', 'shell',
                                 '-a',
                                 'cat /jupyter/users/{default_user}/{student_id}.tsv >> /jupyter/admin/admin_tools/data/students.tsv'.format(default_user=default_user, student_id=student_id)] + target_hub)
    print(r.decode('utf8'))

# JupyterHubユーザーの追加

ローカルユーザーと対応するJupyterHubのユーザーを追加します。

[API Tokens](/hub/token)のページで、tokenを作成して以下に入力してください。

In [ ]:
import getpass
apitoken = getpass.getpass('JupyterHub API token')

In [ ]:
import os
api_url = os.environ['JUPYTERHUB_API_URL']

In [ ]:
import json
import requests

for local_user_info in local_user_infos:
    student_mail = local_user_info.split('\n')[1].split()[0].strip()
    print(student_mail)
    student_id = get_username_from_mail_address(student_mail)
    print(student_id)

    r = requests.get(api_url + '/users/{}'.format(student_id),
                     headers={
                         'Authorization': 'token %s' % apitoken,
                     })
    if r.status_code == 404:
        r = requests.post(api_url + '/users/{}'.format(student_id),
                          headers={
                              'Authorization': 'token %s' % apitoken,
                          })
        assert r.status_code == 201
        print(json.loads(r.text))


# 後始末

一時ファイルを削除する。

In [ ]:
!rm -fr {work_dir}